**IMPORTANTE:** 
- Il runtime di default è **CPU** per non consumare le ore sulle **GPU**, ricordarsi di cambiarlo;

**Collegamento a myDrive per download del dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Clone del repository github**

In [ ]:
!git clone https://github.com/fgiacome/MLDL23-FL-project.git

In [ ]:
import sys
sys.path.append('./MLDL23-FL-project/')
%cd MLDL23-FL-project

**Importazione del dataset**

In [ ]:
# Funzioni utili
import main
from models import deeplabv3, mobilenetv2

# Get dataset (use static variables for args class)
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

df = main.get_datasets(args())

**Verifica del funzionamento del dataset**

Se si vuole verificare il che sia tutto ok prima della fase di training settate **check == True**.

In [ ]:
# Check variable
check = False

# Verifica implementazione __getitem__
# Importazione Numpy e Pyplot 
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import json, os
from datasets.idda import IDDADataset
from datasets import ss_transforms as sstr

if check:

  # The chosen ones
  # constrast 0.6 1.6
  # hue -0.05 .05
  # brightness 0.55 1.6
  # saturation 0.5 1.6

  # Transforms
  train_transforms = sstr.Compose([
              sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6), 
                                saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
              sstr.RandomRotation(degrees = (-5, 5)),
              sstr.PadCenterCrop((925, 1644), fill=255), # 9:16 --> togliere i bordi
              sstr.Resize((512, 928)), # pixel utili per non far impazzire GPU
              sstr.ToTensor(),
              sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225])
          ])
  val_transforms = sstr.Compose([sstr.Resize((512, 928)),
                                 sstr.ToTensor(),
                                  sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])])

  # Import datasets
  # Set random seed (RICORDATEVELO)
  random.seed(300890)

  # codice copiato da get_datasets
  root = 'data/idda'
  client_id = "CENTRALIZED"
  with open(os.path.join(root, 'train.json'), 'r') as f:
      all_data = json.load(f)
  train_img_names = [] # Lista di stringhe da shuffle con metodo nativo random
  for k in all_data.keys():
      for i in all_data[k]:
        train_img_names.append(i)

  # Shuffle data and train-validation splitting
  # random.shuffle(train_img_names)

  # Split in train/validation
  train_size = int(600 * .7)
  val_img_names = train_img_names[train_size:].copy()
  train_img_names = train_img_names[:train_size].copy()

  # Generate train-validation sets
  train_dataset = IDDADataset(root = root, list_samples = train_img_names, 
                              transform = train_transforms, client_name = client_id)

  val_dataset = IDDADataset(root = root, list_samples = val_img_names, 
                            transform = val_transforms, client_name = client_id)

  # Estrazione item 0 del client 0
  img, target = train_dataset.__getitem__(3)
  img_np = img.numpy()
  target_np = target.numpy()

  # Il modulo main normalizza l'immagine
  # Per visualizzarla, la denormalizziamo
  def denormalize(img):
    img[0,:] *= 0.229
    img[1,:] *= 0.224
    img[2,:] *= 0.225
    img[0,:] += 0.485
    img[1,:] += 0.456
    img[2,:] += 0.460
    return img

  # Reshaping per pyplot
  # Pyplot richiede np.array di shape (base, altezza, n_channels)
  def plt_reshape(img):
    img = np.transpose(img, (1, 2, 0))
    return img

  # Plot
  fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 5))
  axs[0].set_title('Image')
  axs[0].imshow(plt_reshape(denormalize(img)))
  axs[0].set_xticks([])
  axs[0].set_yticks([])
  axs[0].set_xticklabels([])
  axs[0].set_yticklabels([])
  axs[1].set_title('Target')
  axs[1].imshow(target_np)
  axs[1].set_xticks([])
  axs[1].set_yticks([])
  axs[1].set_xticklabels([])
  axs[1].set_yticklabels([])

**Training and test function**

In [ ]:
# Useful function for model training and testing
# Import Intersect Over Union
from utils.stream_metrics import StreamSegMetrics

# Training function
def train(net, data_loader, loss_function, optimizer):

  # Loop initialization
  samples = 0
  cumulative_loss = 0
  mean_iou = StreamSegMetrics(n_classes = 16, name = 'Mean IoU')

  # Set the training mode to the model
  net.train()

  # Loop over batches
  for idx, (X, y) in enumerate(data_loader):

    # Send data to GPU
    X = X.cuda()
    # y = y.long()
    y = y.cuda()

    # Predictions
    y_hat = net(X)['out']
    y_pred = torch.argmax(y_hat, dim = 1)

    # Compute loss
    loss = loss_function(y_hat, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Update training variables
    samples += X.size(0)
    cumulative_loss += loss.item() 
    mean_iou.update(y.cpu().numpy(), y_pred.cpu().numpy())

  return cumulative_loss / samples, mean_iou.get_results()



# Test function
def test(net, data_loader, loss_function):

  # Test variables initialization
  samples = 0
  cumulative_loss = 0
  mean_iou = StreamSegMetrics(n_classes = 16, name = 'Mean IoU')

  # Set the evaluation mode to the model
  net.eval()

  # Test loop (we don't want to compute the gradients)
  with torch.no_grad():

    # Loop over batches
    for idx, (X, y) in enumerate(data_loader):

      # Send data to the GPU
      X = X.cuda()
      # y = y.long()
      y = y.cuda()

      # Compute predictions
      y_hat = net(X)['out']
      y_pred = torch.argmax(y_hat, dim = 1)
      
      # Loss computation
      loss = loss_function(y_hat, y)

      # Update test variables
      samples += X.size(0)
      cumulative_loss += loss.item()
      mean_iou.update(y.cpu().numpy(), y_pred.cpu().numpy())

  return cumulative_loss / samples, mean_iou.get_results()

**Iperparametri da testare**:

In funzione dei parametri da testare aggiornare la variable **hyperparameters** a fondo cella.

In [ ]:
###################################################################################
# Comb_list formats: [<Optimizer>, <learning_rate>, <batch_size>, <transforms>] #
###################################################################################

# Alessandro
ale_hyper = {'comb1': ['Adam', 1e-2, 4, True],
             'comb2': ['Adam', 1e-2, 8, True],
             'comb3': ['Adam', 1e-1, 4, True],
             'comb4': ['Adam', 1e-1, 8, True],
             'comb5': ['best_optimizer', 'best_batch_size', False]} # Ultima riga da inserire (BEST COMB + NO TRANSFORMS)

# Davide
dave_hyper = {'comb1': ['Adam', 1e-3, 4, True],
              'comb2': ['Adam', 1e-3, 8, True],
              'comb3': ['SGD', 1e-4, 4, True],
              'comb4': ['SGD', 1e-4, 8, True],
              'comb5': ['Adam', 1e-3, 8, False]} # Ultima riga da inserire (BEST COMB + NO TRANSFORMS)

# Francesco
fra_hyper = {'comb1': ['SGD', 1e-3, 4, True],
             'comb2': ['SGD', 1e-3, 8, True],
             'comb3': ['SGD', 1e-2, 4, True],
             'comb4': ['SGD', 1e-2, 8, True],
             'comb5': ['best_optimizer', 'best_batch_size', False]} # Ultima riga da inserire (BEST COMB + NO TRANSFORMS)

# Hyperparameters 
hyperparameters = dave_hyper

**Training/Test Loop**

In [ ]:
# Importations
import torch
import random
import json, os
from datasets.idda import IDDADataset
from datasets import ss_transforms as sstr

# Model initialization
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

model = main.model_init(args).cuda()

################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
comb = 'comb5' #################################################################
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################

# Transforms
##################################################################################
# Comb_list formats: [<Optimizer>, <learning_rate>, <batch_size>, <transforms>] #
#################### [     0     ,        1       ,       2     ,       3     ] #
##################################################################################

if hyperparameters[comb][3] == True:
  # Transforms
  train_transforms = sstr.Compose([
              sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6), 
                                saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
              sstr.RandomRotation(degrees = (-5, 5)),
              sstr.PadCenterCrop((925, 1644), fill=255), # 9:16 --> togliere i bordi
              sstr.Resize((512, 928)),
              sstr.ToTensor(),
              sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225])
          ])
  val_transforms = sstr.Compose([sstr.Resize((512, 928)),
                                 sstr.ToTensor(),
                                  sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])])
  test_transforms = sstr.Compose([sstr.Resize((512, 928)),
                                  sstr.ToTensor(),
                                  sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])])

else:

  train_transforms = sstr.Compose([
              sstr.Resize((512, 928)),
              sstr.ToTensor(),
              sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
          ])
  val_transforms = sstr.Compose([sstr.Resize((512, 928)),
                                 sstr.ToTensor(),
                                 sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])])
  test_transforms = sstr.Compose([sstr.Resize((512, 928)),
                                  sstr.ToTensor(),
                                  sstr.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])])
  
# Set random seed
random.seed(300890)

# codice copiato da get_datasets
root = 'data/idda'
client_id = "CENTRALIZED"
with open(os.path.join(root, 'train.json'), 'r') as f:
    all_data = json.load(f)
train_img_names = [] # Lista di stringhe da shuffle con metodo nativo random
for k in all_data.keys():
    for i in all_data[k]:
      train_img_names.append(i)

# Shuffle data and train-validation splitting
random.shuffle(train_img_names)

# Split in train/validation
train_size = int(600 * .7)
val_img_names = train_img_names[train_size:].copy()
train_img_names = train_img_names[:train_size].copy()

# Generate train-validation sets
train_dataset = IDDADataset(root = root, list_samples = train_img_names, 
                            transform = train_transforms, client_name = client_id)

val_dataset = IDDADataset(root = root, list_samples = val_img_names, 
                          transform = val_transforms, client_name = client_id)

# Test set
with open(os.path.join(root, 'test_same_dom.txt'), 'r') as f:
    test_same_dom_data = f.read().splitlines()
    test_same_dom_dataset = IDDADataset(root = root, list_samples = test_same_dom_data, 
                                        transform = test_transforms,
                                        client_name='test_same_dom')
with open(os.path.join(root, 'test_diff_dom.txt'), 'r') as f:
    test_diff_dom_data = f.read().splitlines()
    test_diff_dom_dataset = IDDADataset(root=root, list_samples = test_diff_dom_data, 
                                        transform = test_transforms,
                                        client_name='test_diff_dom')

# Loss function
criterion = torch.nn.CrossEntropyLoss(ignore_index = 255)

# Optimizer
##################################################################################
# Comb_list formats: [<Optimizer>, <learning_rate>, <batch_size>, <transforms>] #
#################### [     0     ,        1       ,       2     ,       3     ] #
##################################################################################
if hyperparameters[comb][0] == 'SGD':
  optimizer = torch.optim.SGD(model.parameters(), lr = hyperparameters[comb][1],
                              momentum = .9, weight_decay = 1e-4)
elif hyperparameters[comb][0] == 'Adam':
  optimizer = torch.optim.Adam(model.parameters(), lr = hyperparameters[comb][1])



# Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size = hyperparameters[comb][2],
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 16,
                                           shuffle=True, num_workers=2)
test_same_dom_loader = torch.utils.data.DataLoader(test_same_dom_dataset, 
                                                   batch_size = 16, 
                                                   shuffle=True, num_workers=2)
test_diff_dom_loader = torch.utils.data.DataLoader(test_diff_dom_dataset, 
                                                   batch_size = 16,
                                                   shuffle=True, num_workers=2)

# Maximum number of epochs
epochs = 20

# Training loop
# Loss tracking
train_loss_list = []
train_miou_list = []
val_loss_list = []
val_miou_list = []
test_same_dom_loss_list = []
test_same_dom_miou_list = []
test_diff_dom_loss_list = []
test_diff_dom_miou_list = []

# Epochs loop
for epoch in range(epochs):

  # Training
  train_loss, train_miou = train(model, train_loader, criterion,
                                     optimizer)
  
  # Validation
  val_loss, val_miou = test(model, val_loader, criterion)

  # Evaluation
  test_same_dom_loss, test_same_dom_miou = test(model, test_same_dom_loader, criterion)
  test_diff_dom_loss, test_diff_dom_miou = test(model, test_diff_dom_loader, criterion)

  # Store training and validation losses and accuracies
  train_loss_list.append(train_loss)
  train_miou_list.append(train_miou)
  val_loss_list.append(val_loss)
  val_miou_list.append(val_miou)
  test_same_dom_loss_list.append(test_same_dom_loss)
  test_same_dom_miou_list.append(test_same_dom_miou)
  test_diff_dom_loss_list.append(test_diff_dom_loss)
  test_diff_dom_miou_list.append(test_diff_dom_miou)

  # Display epoch results
  # if (epoch + 1) % 100 == 0:
  print('-----------------------------------------------------')
  print(f'\tEpoch: {epoch + 1}')
  print('\t Training loss {:.5f}, Training Mean IoU {:.2f}'.format(train_loss,
        train_miou['Mean IoU']))
  print('\t Validation loss {:.5f}, Validation Mean IoU {:.2f}'.format(val_loss,
        val_miou['Mean IoU']))
  print('\t Test same dom loss {:.5f}, Test same dom Mean IoU {:.2f}'.format(test_same_dom_loss,
        test_same_dom_miou['Mean IoU']))
  print('\t Test diff dom loss {:.5f}, Test diff dom Mean IoU {:.2f}'.format(test_diff_dom_loss,
        test_diff_dom_miou['Mean IoU']))
  print('-----------------------------------------------------')

In [ ]:
# Import json
import json

# Define file name
filename = 'results'
for hyper in hyperparameters[comb]:
  filename += '_'
  filename += str(hyper) 
filename += '.json'

# Save results on a dict
results_dict = {
                'Train': (train_loss_list, train_miou_list), 
                'Validation': (val_loss_list, val_miou_list),
                'Test - Same Dom': (test_same_dom_loss_list, 
                                    test_same_dom_miou_list),
                'Test - Diff Dom': (test_diff_dom_loss_list, 
                                    test_diff_dom_miou_list)
              }

# Save data on file system (Remember to download it!)
with open(filename, 'w') as fp:
  json.dump(results_dict, fp)